Load libraries.

In [1]:
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import data
import statsmodels.formula.api as sm

Define ticker of interest.

In [2]:
ticker = 'AAPL'

Load historical data.

In [3]:
tickerFile = ticker + '.csv'
data = pd.read_csv(tickerFile, parse_dates=['Date'])
data = data.sort_values(by='Date')
data.set_index('Date', inplace=True)

data['Returns'] = data['Adj Close'].pct_change()  # Create daily returns column
returns = data['Returns'].dropna()  # Remove values of N/A

Load Fama-French data.

In [4]:
factors = pdr.DataReader(name='F-F_Research_Data_5_Factors_2x3_Daily',
                         data_source='famafrench')[0]

Reformat source data.

In [5]:
factors.rename(columns={'Mkt-RF': 'MKT'}, inplace=True)
factors['MKT'] = factors['MKT'] / 100
factors['SMB'] = factors['SMB'] / 100
factors['HML'] = factors['HML'] / 100
factors['RMW'] = factors['RMW'] / 100
factors['CMA'] = factors['CMA'] / 100

Add Fama-French factor data to historical stock data

In [6]:
stock_factor_mat = pd.merge(data,
                            factors,
                            left_index=True,
                            right_index=True)  # Merging the stock and factor returns dataframes together
stock_factor_mat['XsRet'] = (stock_factor_mat['Returns']
                             - stock_factor_mat['RF'])  # Calculating excess returns

Fit Fama-French data to asset pricing model

In [7]:
CAPM = sm.ols(formula='XsRet ~ MKT',
              data=stock_factor_mat).fit()

FF3 = sm.ols(formula='XsRet ~ MKT + SMB + HML',
             data=stock_factor_mat).fit()

FF5 = sm.ols(formula='XsRet ~ MKT + SMB + HML + RMW + CMA',
             data=stock_factor_mat).fit()

Determine t-values and coefficients for asset pricing models

In [8]:
CAPM_tstat = CAPM.tvalues
FF3_tstat = FF3.tvalues
FF5_tstat = FF5.tvalues

CAPM_coeff = CAPM.params
FF3_coeff = FF3.params
FF5_coeff = FF5.params

Consolidate asset pricing model results in a single DataFrame

In [9]:
results = pd.DataFrame({'CAPM_coeff': CAPM_coeff,
                        'CAPM_tstat': CAPM_tstat,
                        'FF3_coeff': FF3_coeff,
                        'FF3_tstat': FF3_tstat,
                        'FF5_coeff': FF5_coeff,
                        'FF5_tstat': FF5_tstat},
                       index=['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])

View asset pricing model data

In [10]:
print(results)

           CAPM_coeff  CAPM_tstat  FF3_coeff  FF3_tstat  FF5_coeff  FF5_tstat
Intercept   -0.003684  -10.847788  -0.003765 -11.404335  -0.003801 -11.904769
MKT          1.209397   30.828436   1.198640  30.945961   1.126850  28.314818
SMB               NaN         NaN  -0.222737  -3.371747  -0.158997  -2.451211
HML               NaN         NaN  -0.454974  -7.611174  -0.091410  -1.252964
RMW               NaN         NaN        NaN        NaN   0.444319   4.570598
CMA               NaN         NaN        NaN        NaN  -1.026216  -8.283938
